In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  %matplotlib inline
else:
    print('Not running on CoLab')
    %matplotlib notebook
    import os
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import time
from os import path

from matplotlib.animation import FFMpegWriter

from funtion_file import *
from ks_funtions import *
from tensorflow.keras.metrics import Precision, Recall

import keras as ks

In [ ]:
row_len = 15
col_len = 20
kernel_size = (2, 2)
min_max_line_size = [(1,5),(1,7)]
rotate = True
num_of_mat = 500
fades_per_mat = 8
num_neuron = 256
new_background = False
triangle = False


matrix_lister = MatrixLister(row_len, col_len, kernel_size, min_max_line_size, rotate, num_of_mat, fades_per_mat, new_background, triangle)

In [ ]:
matrix_lister.unique_lines()

In [ ]:
checkpoint_filepath = 'weights.h5'
model_checkpoint_callback = ks.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
                                                         monitor='loss', mode='min', save_best_only=True)

model = build_model(row_len, col_len, 32, fades_per_mat)
optimizer = ks.optimizers.Adam()
model.compile(optimizer=optimizer, loss=custom_weighted_loss, metrics=[Precision(name='precision'), Recall(name='recall')])

In [ ]:
if path.exists('weights_good_triangle.h5'):
    model.load_weights('weights_good_triangle.h5')
    print('Loaded weights')

In [ ]:
batch_size = 2**11
epochs = 25

generator = DataGenerator(matrix_lister, batch_size, fades_per_mat, num_of_mat)

start = time.time()
hist = model.fit(generator, epochs=epochs, callbacks=[model_checkpoint_callback])

print(time.time() - start)

In [ ]:
model.save_weights('weights_good_triangle.h5')

In [ ]:
ani = matrix_lister.plot_matrices(model, 3, interval=500)
plt.show()

In [ ]:
if 'google.colab' in str(get_ipython()):
    f = r"animation.mp4"
    writervideo = FFMpegWriter(fps=24)
    ani.save(f, writer=writervideo)

In [ ]:
plot_training_history(hist, with_val=False)